In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import time
import datetime 
%matplotlib inline  
import sys
sys.path.append('/gpfs2/projects/project-bus_capstone_2016/workspace/jz2308/bus-Capstone')

# these two modules are homemade
import gtfs
import arrivals
import ttools
os.chdir('/gpfs2/projects/project-bus_capstone_2016/workspace/share')

In [2]:
base = datetime.date(2015, 1, 1)
schedule_samples = [str(base + datetime.timedelta(days=x)) for x in range(0, 365)]
#schedule_samples = ['2015-01-04','2015-04-05','2015-06-27','2015-07-06','2015-09-05','2015-09-15','2015-10-12']
collected = pd.DataFrame()
for sdate in schedule_samples[0:1]:
    stops = gtfs.load_stops(sdate,'gtfs/')
    stops['schedule_sample_date'] = sdate
    collected = collected.append(stops[['stop_lat','stop_lon','stop_name','schedule_sample_date']].drop_duplicates())

In [3]:
collected.reset_index().head()

,stop_id,stop_lat,stop_lon,stop_name,schedule_sample_date
0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01
1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01
2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01
3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01
4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01


In [5]:
collected.reset_index().head()

,stop_id,stop_lat,stop_lon,stop_name,schedule_sample_date
0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01
1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01
2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01
3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01
4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01


In [7]:
reliability_metrics = pd.read_csv('performance.csv',names = ["Line", "stop_id", "schedule_sample_date", "ontime_ratio", "peak_hours wait ass","offpeak hours wait ass"])

In [8]:
reliability_metrics.head()

,Line,stop_id,schedule_sample_date,ontime_ratio,peak_hours wait ass,offpeak hours wait ass
0,Q07,350232,2015-01-02,0.831395,0.89156626506,1.000000
1,Q25,550032,2015-01-02,0.520737,0.723880597015,0.760369
2,Q72,552020,2015-01-01,1.000000,None,1.000000
3,Q47,551547,2015-01-02,0.000000,1.0,0.000000
4,Q66,500994,2015-01-30,0.486301,0.601694915254,0.958904


In [9]:
pd.DataFrame(reliability_metrics.reset_index().groupby('stop_id').mean().reset_index()).head()

,stop_id,index,ontime_ratio,offpeak hours wait ass
0,100014,2228436,0.139963,0.921871
1,100017,3101092,0.210164,0.913365
2,100018,3631244,0.177457,0.892479
3,100019,3116986,0.203798,0.918004
4,100020,2587259,0.203381,0.854682


In [10]:
stops_metrics = collected.reset_index().merge(pd.DataFrame(reliability_metrics.reset_index().groupby('stop_id').mean().reset_index()))
stops_metrics = stops_metrics.groupby('stop_id').mean()[['stop_lat','stop_lon','ontime_ratio','offpeak hours wait ass']]

In [12]:
stops_metrics.head()

,stop_lat,stop_lon,ontime_ratio,offpeak hours wait ass
stop_id,,,,
100014,40.872562,-73.888153,0.139963,0.921871
100017,40.876808,-73.889656,0.210164,0.913365
100018,40.880341,-73.886063,0.177457,0.892479
100019,40.813496,-73.929489,0.203798,0.918004
100020,40.816803,-73.927956,0.203381,0.854682


In [13]:
stops_metrics.to_csv('metrics-by-stop-2015.csv')

In [14]:
# get all the schedule data. (subset can be created later)
trips = gtfs.load_trips('2015-12-03','gtfs/')
# stops = gtfs.load_stops('2015-12-03','gtfs/')
stop_times, tz_sched = gtfs.load_stop_times('2015-12-03','gtfs/')
print 'Finished loading GTFS data.'
joined = stop_times.reset_index(level=1)[['stop_id','stop_sequence']].join(trips[['route_id','direction_id']])
unique_stops = joined.reset_index().drop_duplicates(subset=['route_id','direction_id','stop_id','stop_sequence']).drop('trip_id',axis=1)
unique_stops = unique_stops.set_index(['route_id','stop_id'],verify_integrity=False)
dupe_inds = unique_stops.index[unique_stops.index.duplicated()]
unique_stops = unique_stops.loc[dupe_inds]

Finished loading GTFS data.


In [15]:
collected = collected.reset_index().join(unique_stops.reset_index(),how='left',lsuffix='_left', rsuffix='_right')

In [16]:
collected.head()

,stop_id_left,stop_lat,stop_lon,stop_name,schedule_sample_date,route_id,stop_id_right,stop_sequence,direction_id
0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01,BX23,150139,24,1
1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01,BX23,150139,1,0
2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01,BX23,150139,2,1
3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01,BX23,150038,1,1
4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01,BX23,150038,23,0


In [19]:
line_metrics = collected.reset_index().join((pd.DataFrame(reliability_metrics.groupby(['Line','stop_id']).mean().reset_index())),how='left',lsuffix='_left', rsuffix='_right')

In [22]:
line_metrics.head()

,index,stop_id_left,stop_lat,stop_lon,stop_name,schedule_sample_date,route_id,stop_id_right,stop_sequence,direction_id,Line,stop_id,ontime_ratio,offpeak hours wait ass
0,0,300000,40.578209,-73.939697,ORIENTAL BL/MACKENZIE ST,2015-01-01,BX23,150139,24,1,B1,300000,0.153516,0.921430
1,1,300002,40.578068,-73.943031,ORIENTAL BL/JAFFRAY ST,2015-01-01,BX23,150139,1,0,B1,300002,0.154064,0.889592
2,2,300003,40.577961,-73.944664,ORIENTAL BL/HASTINGS ST,2015-01-01,BX23,150139,2,1,B1,300003,0.226618,0.900917
3,3,300004,40.577721,-73.946205,ORIENTAL BL/FALMOUTH ST,2015-01-01,BX23,150038,1,1,B1,300004,0.226554,0.918115
4,4,300006,40.577354,-73.949554,ORIENTAL BL/DOVER ST,2015-01-01,BX23,150038,23,0,B1,300006,0.202300,0.917007


In [23]:
line_metrics.query("Line=='B41'")#.to_csv('metrics-by-line-stop-B41-2015.csv')

,index,stop_id_left,stop_lat,stop_lon,stop_name,schedule_sample_date,route_id,stop_id_right,stop_sequence,direction_id,Line,stop_id,ontime_ratio,offpeak hours wait ass
2085,2085,303849,40.724411,-73.949364,NASSAU AV/ECKFORD ST,2015-01-01,QM5,502361,40,0,B41,300163,0.176039,0.888042
2086,2086,303850,40.723892,-73.950668,NASSAU AV/MANHATTAN AV,2015-01-01,QM5,502361,37,0,B41,302434,0.063032,0.908949
2087,2087,303852,40.719131,-73.950302,LORIMER ST/BAYARD ST,2015-01-01,QM5,502363,44,0,B41,302623,0.121606,0.957090
2088,2088,303853,40.716995,-73.949921,LORIMER ST/MEEKER AV,2015-01-01,QM5,502363,46,0,B41,303215,0.203655,0.916641
2089,2089,303854,40.715157,-73.949623,LORIMER ST/SKILLMAN AV,2015-01-01,QM5,502363,41,0,B41,303218,0.179740,0.874663
2090,2090,303855,40.713718,-73.949387,LORIMER ST/METROPOLITAN AV,2015-01-01,QM5,502363,38,0,B41,303219,0.169476,0.909057
2091,2091,303856,40.712322,-73.949158,LORIMER ST/AINSLIE ST,2015-01-01,QM5,502364,45,0,B41,303222,0.178442,0.897610
2092,2092,303857,40.711025,-73.948944,LORIMER ST/GRAND ST,2015-01-01,QM5,502364,47,0,B41,303223,0.183056,0.866537
2093,2093,303858,40.709419,-73.948677,LORIMER ST/TEN EYCK ST,2015-01-01,QM5,502364,42,0,B41,303224,0.168804,0.878831
2094,2094,303859,40.707287,-73.948326,LORIMER ST/MESEROLE ST,2015-01-01,QM5,502364,39,0,B41,303227,0.169452,0.862365


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,20))
plt.scatter(stops_metrics.stop_lon,stops_metrics.stop_lat,c=stops_metrics.ontime_ratio,s=55)
